In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import time
from tqdm import trange
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import gym
import pybullet as p
import stage.envs
from stage.tasks.twolink.reaching import TwoLinkReaching
from stage.utils.nn import use_gpu
use_gpu()

In [4]:
savepath = parentdir + '/data/twolink/'

In [5]:
from stage.utils.nn import bquad, bmv

In [6]:
task = TwoLinkReaching(render=False)

# Read task parameters, should we use a separate class to wrap this?

nq, nv, nu, nx = task.nq, task.nv, task.nu, task.nx
dt_control, dt_env = task.dt_control, task.dt_env
q_lb, q_ub = task.q_lb, task.q_ub

In [7]:
# Setup action parameterization

from stage.controllers.actor import Actor
from stage.controllers.trivial import Identity
from stage.controllers.pd import PD

# na = 2
# action_ub = torch.Tensor(task.env.action_space.high)
# action_lb = torch.Tensor(task.env.action_space.low)
# actor = Actor(na, Identity(nq, nv, nu), action_lb, action_ub)

na = 4
gain_ub = 50 * torch.ones((nq))
gain_lb = 0. * torch.ones((nq))
action_ub = torch.cat((gain_ub, q_ub))
action_lb = torch.cat((gain_lb, q_lb))
actor = Actor(na, PD(nq, nv, nu), action_lb, action_ub)

# this is extremely ugly
task.cost.actor = actor

In [8]:
# Setup model learning

from stage.dynamics.mlp import MLPDyn, DefaultDx
dynamics = MLPDyn(nx, nq, nv, na, dt_control, DefaultDx)

# from stage.dynamics.probabilistic_ensemble import ProbabilisticEnsemble, DefaultDx
# ensemble_size = 5
# dynamics = ProbabilisticEnsemble(nq, nv, na, dt_control, 
#                                  DefaultDx,
#                                  ensemble_size)

In [9]:
# Setup controller
# from stage.controllers.trivial import RandomController

# controller = RandomController(nx, actor)

from stage.controllers.ilqr import ILQR
controller = ILQR(dynamics, task.cost, actor, task.task_horizon)

In [10]:
# Setup learner

from stage.learners.learn_and_control_model import LearnAndControlModel
learner = LearnAndControlModel(task, dynamics, controller)

In [11]:
_ = learner.learn(10, verbose=True)
learner.save_training_data(savepath+'data_pd_reg')

Network training: 100%|██████████| 10/10 [00:00<00:00, 20.69epoch(s)/s, Training loss MSE=2.183558]


Iteration:  0
avg. decision time:  0.00447084903717041
obs. reward:  -258.5878
act. reward:  -1.4069816


/home/hz/ws/stage/src/stage/controllers/ilqr.py:126: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")
/home/hz/ws/stage/src/stage/controllers/ilqr.py:119: UserWarning: ill-conditioned Qaa
  warnings.warn("ill-conditioned Qaa")
Network training: 100%|██████████| 10/10 [00:01<00:00,  6.47epoch(s)/s, Training loss MSE=nan]


Iteration:  1
avg. decision time:  1.1216005325317382
obs. reward:  nan
act. reward:  nan


KeyboardInterrupt: 

In [ ]:
traj, log = task.perform(task.goal, controller)
act_seq = traj[:, nx:nx+na]
initial_obs = traj[0, :nx]
final_obs = traj[-1, :nx]

In [ ]:
n_sample = 50
traj_pred = dynamics.unroll(initial_obs, 
                            act_seq, 
                            n_sample)

predicted_err = torch.norm(traj_pred[-1, :, :]-final_obs.expand(n_sample, -1), p=2, dim=1)
print (predicted_err.mean())
print (predicted_err.std())

In [ ]:
from stage.utils.nn import jacobian_vector_product

In [ ]:
x = initial_obs.clone()
a = act_seq[0].clone()

In [ ]:
# cost = task.cost.l(x, a, diff=True)

In [ ]:
# from stage.controllers.ilqr import ILQR
# ilqr = ILQR(dynamics, task.cost, actor, task.task_horizon)
# rollout, sols = ilqr.optimize(initial_obs)

In [ ]:
# from stage.controllers.trivial import OpenLoop
# openloop = OpenLoop(nx, actor, actions)

In [ ]:
task_horizon = task.task_horizon

traj_pred_mean = torch.mean(traj_pred, dim=1)
traj_pred_std = torch.std(traj_pred, dim=1)
traj_pred_mean = traj_pred_mean.detach().cpu().numpy()
traj_pred_std = traj_pred_std.detach().cpu().numpy()
traj = traj.detach().cpu().numpy()

desired = task.cost.desired.repeat((task_horizon, 1))
desired = desired.detach().cpu().numpy()

In [ ]:
font = {'family' : 'serif',
        'size'   : 22}

matplotlib.rc('font', **font)
np.set_printoptions(precision=3, linewidth=200, suppress=True)

In [ ]:
# traj_pred_mean = np.load('traj_pred_mean.npy')
# traj_pred_std = np.load('traj_pred_std.npy')
# traj = np.load('traj.npy')

In [ ]:
d = nq
dt = dt_control
fig, ax = plt.subplots(d, figsize=(10, d * 6))
t = np.arange(0.0, task_horizon*dt, dt)
lb = traj_pred_mean - traj_pred_std
ub = traj_pred_mean + traj_pred_std

for i in range(d):
    ax[i].plot(t, traj[:, i], lw=4, color='orange', label='actual')
    ax[i].plot(t, traj_pred_mean[:, i], lw=4, color='b', label='predicted mean')
#     ax[i].plot(t, desired[:, i], lw=2, color='k', ls='-.', label='desired')
    lb = traj_pred_mean - traj_pred_std
    ub = traj_pred_mean + traj_pred_std
    ax[i].fill_between(t, lb[:, i], ub[:, i], facecolor='blue',
                alpha=0.2)
    _ = ax[i].grid()
    _ = ax[i].set_ylim([-3.2, 3.2])
    ax[i].legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
             ncol=3, fancybox=True, shadow=True)
# fig.savefig('prediction_with_reg_150steps' + '.png', bbox_inches='tight')